In [21]:
# KB차차차 중고차 매물 크롤링 - 페이지별 carSeq 추출
import requests
import json
import pandas as pd
from datetime import datetime
import time
from bs4 import BeautifulSoup
import re

BASE_URL = "https://www.kbchachacha.com"

session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36',
    'Accept': '*/*',
    'Accept-Language': 'ko,ko-KR;q=0.9,en-US;q=0.8,en;q=0.7',
    'Referer': 'https://www.kbchachacha.com/public/search/main.kbc',
    'Host': 'www.kbchachacha.com'
})

def get_car_seqs_from_page(page_num):
    url = f"https://www.kbchachacha.com/public/search/list.empty?page={page_num}&sort=-orderDate"
    
    try:
        res = session.get(url, timeout=10)
        if res.status_code == 200:
            soup = BeautifulSoup(res.text, 'html.parser')
            
            # .area 클래스에서 data-car-seq 추출
            areas = soup.select('.area')
            page_car_seqs = []
            
            for area in areas:
                car_seq = area.get('data-car-seq')
                if car_seq:
                    page_car_seqs.append(car_seq)
            
            print(f"페이지 {page_num}: {len(page_car_seqs)}개 carSeq 발견")
            return page_car_seqs
        else:
            print(f"페이지 {page_num}: HTTP {res.status_code}")
            return []
    except Exception as e:
        print(f"페이지 {page_num} 오류: {e}")
        return []

def crawl_multiple_pages(max_pages=5, delay=1):
    """여러 페이지를 크롤링하여 모든 carSeq를 수집합니다."""
    all_car_seqs = []
    
    print(f"KB차차차 {max_pages}페이지 크롤링 시작...")
    
    for page in range(1, max_pages + 1):
        print(f"\n페이지 {page} 처리 중...")
        
        page_car_seqs = get_car_seqs_from_page(page)
        
        if page_car_seqs:
            all_car_seqs.extend(page_car_seqs)
            print(f"페이지 {page} carSeq: {page_car_seqs[:5]}...")  # 처음 5개만 출력
        else:
            print(f"페이지 {page}: carSeq 없음")
            break  # 더 이상 데이터가 없으면 중단
        
        # 서버 부하 방지
        time.sleep(delay)
    
    # 중복 제거
    unique_car_seqs = list(set(all_car_seqs))
    
    print(f"\n=== 크롤링 완료 ===")
    print(f"총 수집된 carSeq: {len(all_car_seqs)}개")
    print(f"중복 제거 후: {len(unique_car_seqs)}개")
    print(f"carSeq 목록: {unique_car_seqs[:10]}...")  # 처음 10개만 출력
    
    return unique_car_seqs

# 크롤링 실행 (테스트용으로 3페이지)
print("KB차차차 페이지별 carSeq 크롤링 시작...")
all_car_seqs = crawl_multiple_pages(max_pages=3, delay=2)

KB차차차 페이지별 carSeq 크롤링 시작...
KB차차차 3페이지 크롤링 시작...

페이지 1 처리 중...
페이지 1: 40개 carSeq 발견
페이지 1 carSeq: ['27487656', '27484590', '27463815', '27463321', '27470105']...

페이지 2 처리 중...
페이지 2: 40개 carSeq 발견
페이지 2 carSeq: ['27249358', '27300433', '27292767', '27287926', '27286321']...

페이지 3 처리 중...
페이지 3: 40개 carSeq 발견
페이지 3 carSeq: ['27487583', '27363782', '27035910', '27481107', '27254066']...

=== 크롤링 완료 ===
총 수집된 carSeq: 120개
중복 제거 후: 120개
carSeq 목록: ['27369166', '27424012', '27248418', '27286326', '27411564', '27469637', '27384566', '27481107', '27452275', '27338086']...


In [26]:
# 🚀 더 효율적인 방법: API를 통한 일괄 정보 수집
def get_car_info_via_api(car_seqs):
    """carSeq 리스트를 API로 보내서 일괄로 차량 정보를 받아옵니다."""
    
    # API 엔드포인트
    api_url = "https://www.kbchachacha.com/public/car/common/recent/car/list.json"
    
    # carSeq를 쉼표로 구분된 문자열로 변환
    car_seq_str = ",".join(car_seqs)
    
    # POST 요청 데이터
    payload = {
        'gotoPage': 1,
        'pageSize': len(car_seqs),
        'carSeqVal': car_seq_str
    }
    
    print(f"\n=== API 요청 정보 ===")
    print(f"URL: {api_url}")
    print(f"carSeq 개수: {len(car_seqs)}")
    print(f"carSeqVal: {car_seq_str}")
    
    try:
        # POST 요청
        response = session.post(api_url, data=payload, timeout=10)
        
        print(f"HTTP 상태코드: {response.status_code}")
        print(f"응답 크기: {len(response.text)} bytes")
        
        if response.status_code == 200:
            # JSON 파싱
            data = response.json()
            
            print(f"✅ API 응답 성공!")
            print(f"총 차량 수: {data.get('totalCount', 0)}")
            print(f"실제 리스트 수: {len(data.get('list', []))}")
            
            return data.get('list', [])
        else:
            print(f"❌ HTTP 오류: {response.status_code}")
            return []
            
    except Exception as e:
        print(f"❌ API 요청 실패: {e}")
        return []

# 테스트: API를 통한 차량 정보 수집
if all_car_seqs:
    print(f"\n{'='*60}")
    print("🚀 API를 통한 효율적인 차량 정보 수집 테스트")
    print(f"{'='*60}")
    
    car_info_list = get_car_info_via_api(all_car_seqs)
    
    if car_info_list:
        print(f"\n=== 수집된 차량 정보 ===")
        for i, car in enumerate(car_info_list, 1):
            print(f"\n{i}. {car.get('makerName', '')} {car.get('carName', '')} {car.get('modelName', '')}")
            print(f"   carSeq: {car.get('carSeq')}")
            print(f"   가격: {car.get('sellAmt', 0):,}만원")
            print(f"   연식: {car.get('yymm', '')}년")
            print(f"   주행거리: {car.get('km', 0):,}km")
            print(f"   지역: {car.get('cityName', '')}")
            print(f"   차량번호: {car.get('carNo', '')}")
            print(f"   연료: {car.get('gasName', '')}")
            print(f"   사고이력: {car.get('accidentNo', '')}")
            print(f"   담보: {car.get('distraintInfo', 0)}")
            print(f"   저당: {car.get('mortgageInfo', 0)}")
            print(f"   세금체납: {car.get('taxDefaultInfo', 0)}")
    else:
        print("❌ API를 통한 정보 수집에 실패했습니다.")



🚀 API를 통한 효율적인 차량 정보 수집 테스트

=== API 요청 정보 ===
URL: https://www.kbchachacha.com/public/car/common/recent/car/list.json
carSeq 개수: 120
carSeqVal: 27369166,27424012,27248418,27286326,27411564,27469637,27384566,27481107,27452275,27338086,27287926,27097754,27157707,26997494,27397686,27448244,27487583,27469379,27379260,27487783,27443985,27154237,27368019,27489256,27469408,27096874,27443975,27444360,27416994,27477575,26575964,26970934,27470105,27399152,27408437,27056673,27182826,27320111,27120452,27338517,27229911,27447421,27390016,26962413,27500221,27468471,27345936,27413726,27463309,27463815,27154135,27363782,27035910,27407601,27364907,27487656,27379810,27263569,27292767,27318879,27365658,27189408,27444363,27300433,27462800,27348758,27103154,26908015,27286321,26907562,27254066,27198183,27417353,27424107,27372675,27484590,27102760,27476884,27287243,27493810,27250639,27386574,27346477,27464593,27463321,27369065,27375036,27028843,27433873,27345737,27021712,27466706,27085349,27416871,27236672

In [24]:
def complete_kb_chachacha_crawling(max_pages=3, cars_per_batch=30):
    """KB차차차 완전 크롤링 시스템"""
    
    print(f"🚀 KB차차차 완전 크롤링 시작!")
    print(f"📄 크롤링할 페이지: {max_pages}페이지")
    print(f"📦 배치 크기: {cars_per_batch}개씩")
    print(f"{'='*60}")
    
    all_car_seqs = []
    all_car_data = []
    
    # 1단계: 모든 페이지에서 carSeq 수집
    print(f"\n📋 1단계: carSeq 수집")
    for page in range(1, max_pages + 1):
        print(f"\n페이지 {page} 처리 중...")
        
        url = f"https://www.kbchachacha.com/public/search/list.empty?page={page}&sort=-orderDate"
        
        try:
            res = session.get(url, timeout=10)
            if res.status_code == 200:
                soup = BeautifulSoup(res.text, 'html.parser')
                areas = soup.select('.area')
                
                page_car_seqs = []
                for area in areas:
                    car_seq = area.get('data-car-seq')
                    if car_seq:
                        page_car_seqs.append(car_seq)
                
                all_car_seqs.extend(page_car_seqs)
                print(f"   ✅ {len(page_car_seqs)}개 carSeq 수집")
            else:
                print(f"   ❌ HTTP 오류: {res.status_code}")
                break
                
        except Exception as e:
            print(f"   ❌ 오류: {e}")
            break
        
        time.sleep(1)  # 서버 부하 방지
    
    # 중복 제거
    unique_car_seqs = list(set(all_car_seqs))
    print(f"\n📊 carSeq 수집 완료: 총 {len(all_car_seqs)}개 → 중복 제거 후 {len(unique_car_seqs)}개")
    
    if not unique_car_seqs:
        print("❌ carSeq를 수집할 수 없습니다.")
        return []
    
    # 2단계: 배치 단위로 API 호출
    print(f"\n🔄 2단계: API를 통한 차량 정보 수집")
    
    for i in range(0, len(unique_car_seqs), cars_per_batch):
        batch_car_seqs = unique_car_seqs[i:i + cars_per_batch]
        batch_num = (i // cars_per_batch) + 1
        total_batches = (len(unique_car_seqs) + cars_per_batch - 1) // cars_per_batch
        
        print(f"\n배치 {batch_num}/{total_batches} 처리 중... ({len(batch_car_seqs)}개 차량)")
        
        # API 호출
        batch_data = get_car_info_via_api(batch_car_seqs)
        
        if batch_data:
            all_car_data.extend(batch_data)
            print(f"   ✅ {len(batch_data)}개 차량 정보 수집 완료")
        else:
            print(f"   ❌ 배치 {batch_num} 실패")
        
        # 서버 부하 방지
        if batch_num < total_batches:
            time.sleep(2)
    
    # 3단계: 결과 요약
    print(f"\n{'='*60}")
    print(f"🎉 크롤링 완료!")
    print(f"📊 수집된 차량 수: {len(all_car_data)}개")
    print(f"📄 처리한 페이지: {max_pages}페이지")
    print(f"🔢 총 carSeq: {len(unique_car_seqs)}개")
    
    if all_car_data:
        print(f"\n📋 수집된 차량 정보 샘플:")
        for i, car in enumerate(all_car_data[:3], 1):  # 처음 3개만 출력
            print(f"{i}. {car.get('makerName', '')} {car.get('carName', '')} - {car.get('sellAmt', 0):,}만원")
    
    return all_car_data

# 완전한 크롤링 실행 (테스트용으로 2페이지, 10개씩 배치)
print("🚀 KB차차차 완전 크롤링 시스템 테스트 시작!")
crawled_data = complete_kb_chachacha_crawling(max_pages=2, cars_per_batch=10)


🚀 KB차차차 완전 크롤링 시스템 테스트 시작!
🚀 KB차차차 완전 크롤링 시작!
📄 크롤링할 페이지: 2페이지
📦 배치 크기: 10개씩

📋 1단계: carSeq 수집

페이지 1 처리 중...
   ✅ 40개 carSeq 수집

페이지 2 처리 중...
   ✅ 40개 carSeq 수집

📊 carSeq 수집 완료: 총 80개 → 중복 제거 후 80개

🔄 2단계: API를 통한 차량 정보 수집

배치 1/8 처리 중... (10개 차량)

=== API 요청 정보 ===
URL: https://www.kbchachacha.com/public/car/common/recent/car/list.json
carSeq 개수: 10
carSeqVal: 27490092,27483488,27471263,27463770,27030843,26883495,27466101,27438496,27181931,27328174
HTTP 상태코드: 200
응답 크기: 33272 bytes
✅ API 응답 성공!
총 차량 수: 10
실제 리스트 수: 10
   ✅ 10개 차량 정보 수집 완료

배치 2/8 처리 중... (10개 차량)

=== API 요청 정보 ===
URL: https://www.kbchachacha.com/public/car/common/recent/car/list.json
carSeq 개수: 10
carSeqVal: 27468103,26845833,27485347,27442509,26846611,27492997,27454859,27452008,27485739,27432903
HTTP 상태코드: 200
응답 크기: 33249 bytes
✅ API 응답 성공!
총 차량 수: 10
실제 리스트 수: 10
   ✅ 10개 차량 정보 수집 완료

배치 3/8 처리 중... (10개 차량)

=== API 요청 정보 ===
URL: https://www.kbchachacha.com/public/car/common/recent/car/list.json
carSeq 개수: 10

In [ ]:
# 📁 데이터 저장 및 분석
def save_and_analyze_data(car_data, filename_prefix="kb_chachacha_complete"):
    """크롤링한 데이터를 저장하고 분석합니다."""
    
    if not car_data:
        print("❌ 저장할 데이터가 없습니다.")
        return
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. CSV 저장
    csv_filename = f"{filename_prefix}_{timestamp}.csv"
    df = pd.DataFrame(car_data)
    df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    print(f"✅ CSV 저장 완료: {csv_filename}")
    
    # 2. JSON 저장
    json_filename = f"{filename_prefix}_{timestamp}.json"
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(car_data, f, ensure_ascii=False, indent=2)
    print(f"✅ JSON 저장 완료: {json_filename}")
    
    # 3. 데이터 분석
    print(f"\n📊 데이터 분석 결과:")
    print(f"총 차량 수: {len(car_data)}개")
    
    # 제조사별 통계
    maker_counts = df['makerName'].value_counts()
    print(f"\n🏭 제조사별 차량 수:")
    for maker, count in maker_counts.head(5).items():
        print(f"  {maker}: {count}개")
    
    # 가격대별 통계
    price_stats = df['sellAmt'].describe()
    print(f"\n💰 가격 통계:")
    print(f"  평균: {price_stats['mean']:,.0f}만원")
    print(f"  최저: {price_stats['min']:,.0f}만원")
    print(f"  최고: {price_stats['max']:,.0f}만원")
    print(f"  중간값: {price_stats['50%']:,.0f}만원")
    
    # 지역별 통계
    city_counts = df['cityName'].value_counts()
    print(f"\n📍 지역별 차량 수:")
    for city, count in city_counts.head(5).items():
        print(f"  {city}: {count}개")
    
    # 연식별 통계
    year_counts = df['yymm'].value_counts().sort_index()
    print(f"\n📅 연식별 차량 수:")
    for year, count in year_counts.head(5).items():
        print(f"  {year}년: {count}개")
    
    return df

# 데이터 저장 및 분석 실행
if 'crawled_data' in locals() and crawled_data:
    df = save_and_analyze_data(crawled_data)
else:
    print("❌ 크롤링된 데이터가 없습니다.")


In [22]:
# carSeq를 이용한 상세 페이지 크롤링 함수
def get_car_detail_by_seq(car_seq):
    """carSeq를 이용해서 상세 페이지 정보를 가져옵니다."""
    detail_url = f"https://www.kbchachacha.com/public/car/detail.kbc?carSeq={car_seq}"
    
    try:
        response = session.get(detail_url, timeout=10)
        if response.status_code == 200:
            return response.text
        else:
            print(f"상세 페이지 오류 (carSeq: {car_seq}): HTTP {response.status_code}")
            return None
    except Exception as e:
        print(f"상세 페이지 요청 실패 (carSeq: {car_seq}): {e}")
        return None

def parse_car_detail(detail_html, car_seq):
    """상세 페이지 HTML에서 차량 정보를 파싱합니다."""
    if not detail_html:
        return None
    
    soup = BeautifulSoup(detail_html, 'html.parser')
    car_info = {
        'car_seq': car_seq,
        'scraped_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'source': 'KB차차차_상세페이지'
    }
    
    try:
        text_content = soup.get_text()
        
        # 차량명 추출
        title_element = soup.find('strong', class_='tit') or soup.find('h1') or soup.find('title')
        if title_element:
            car_info['car_name'] = title_element.get_text(strip=True)
        
        # 가격 추출
        price_patterns = [
            r'(\d{1,3}(?:,\d{3})*만원)',
            r'판매가격[^\d]*(\d{1,3}(?:,\d{3})*만원)',
            r'(\d{1,3}(?:,\d{3})*원)'
        ]
        
        for pattern in price_patterns:
            match = re.search(pattern, text_content)
            if match:
                car_info['price'] = match.group(1)
                break
        
        # 연식 추출
        year_pattern = r'(\d{2}/\d{2}식\(\d{2}년형\))'
        year_match = re.search(year_pattern, text_content)
        if year_match:
            car_info['year'] = year_match.group(1)
        
        # 주행거리 추출
        mileage_pattern = r'(\d{1,3}(?:,\d{3})*km)'
        mileage_match = re.search(mileage_pattern, text_content)
        if mileage_match:
            car_info['mileage'] = mileage_match.group(1)
        
        # 지역 추출
        location_pattern = r'(울산|경기|인천|서울|부산|대구|광주|대전|세종|강원|충북|충남|전북|전남|경북|경남)'
        location_match = re.search(location_pattern, text_content)
        if location_match:
            car_info['location'] = location_match.group(1)
        
        # 연료 타입 추출
        fuel_patterns = [
            r'(가솔린|디젤|LPG|하이브리드|전기|EV)',
            r'(LPe|GDI|T-GDI|TDI)'
        ]
        
        for pattern in fuel_patterns:
            fuel_match = re.search(pattern, text_content)
            if fuel_match:
                car_info['fuel_type'] = fuel_match.group(1)
                break
        
        # 특별 옵션/상태 추출
        feature_patterns = [
            r'(무사고|1인소유|제조사보증|비흡연|특옵션|가격저렴|짧은㎞|완전무사고)',
            r'(KB보증|헛걸음보상|진단홈배송|인증|KB진단)'
        ]
        
        features = []
        for pattern in feature_patterns:
            matches = re.findall(pattern, text_content)
            features.extend(matches)
        
        if features:
            car_info['features'] = list(set(features))  # 중복 제거
        
        # 상세 페이지 URL
        car_info['detail_url'] = f"https://www.kbchachacha.com/public/car/detail.kbc?carSeq={car_seq}"
        
        return car_info
        
    except Exception as e:
        print(f"상세 페이지 파싱 오류 (carSeq: {car_seq}): {e}")
        return None

# 상세 페이지 크롤링 테스트 (처음 3개 차량만)
if all_car_seqs:
    print(f"\n=== 상세 페이지 크롤링 테스트 (처음 3개) ===")
    detailed_cars = []
    
    for i, car_seq in enumerate(all_car_seqs[:3]):  # 처음 3개만 테스트
        print(f"\n{i+1}. carSeq {car_seq} 처리 중...")
        
        # 상세 페이지 가져오기
        detail_html = get_car_detail_by_seq(car_seq)
        
        if detail_html:
            # 상세 페이지 파싱
            car_info = parse_car_detail(detail_html, car_seq)
            
            if car_info:
                detailed_cars.append(car_info)
                print(f"   ✅ 성공: {car_info.get('car_name', 'N/A')} - {car_info.get('price', 'N/A')}")
            else:
                print(f"   ❌ 파싱 실패")
        else:
            print(f"   ❌ 상세 페이지 로드 실패")
        
        # 서버 부하 방지
        time.sleep(1)
    
    print(f"\n상세 페이지 크롤링 결과: {len(detailed_cars)}개 성공")
else:
    print("carSeq가 없어서 상세 페이지 크롤링을 할 수 없습니다.")



=== 상세 페이지 크롤링 테스트 (처음 3개) ===

1. carSeq 27369166 처리 중...
   ✅ 성공: KB차차차 - 2,990만원

2. carSeq 27424012 처리 중...
   ✅ 성공: KB차차차 - 6,050만원

3. carSeq 27248418 처리 중...
   ✅ 성공: KB차차차 - 5,400만원

상세 페이지 크롤링 결과: 3개 성공


## 🔗 M-park 성능점검기록부 연동

상세페이지 HTML에서 `checkNo`를 추출해 M-park API로 성능점검기록부를 조회하고 결과를 병합합니다.


In [ ]:
import urllib.parse

MPARK_CHECK_BASE = "https://api.m-park.co.kr/home/api/v1/wb/searchmycar/carcheckdetailinfo/get"


def extract_mpark_checkno_from_html(detail_html: str):
    """상세 HTML에서 M-park 성능점검 `checkNo`를 추출합니다.
    - 보통 링크/스크립트 쿼리스트링에 `checkNo=...` 형태로 포함됩니다.
    """
    if not detail_html:
        return None
    soup = BeautifulSoup(detail_html, 'html.parser')
    
    # 1) href 내 쿼리스트링에서 checkNo 탐색
    for a in soup.find_all('a', href=True):
        href = a['href']
        if 'checkNo=' in href:
            qs = urllib.parse.urlparse(href).query
            params = urllib.parse.parse_qs(qs)
            if 'checkNo' in params and params['checkNo']:
                return params['checkNo'][0]
    
    # 2) 스크립트/데이터 속 텍스트에서 checkNo=숫자 패턴 탐색
    import re
    m = re.search(r"checkNo=([0-9]{8,})", detail_html)
    if m:
        return m.group(1)
    
    return None


def fetch_mpark_check_report(check_no: str):
    """M-park API로 성능점검기록부 조회"""
    if not check_no:
        return None
    params = {"checkNo": check_no}
    r = session.get(MPARK_CHECK_BASE, params=params, timeout=10)
    if r.status_code != 200:
        return None
    try:
        data = r.json()
        if data.get("statusCode") == 0 and data.get("data"):
            return data["data"][0]
    except Exception:
        return None
    return None


def enrich_with_mpark_report(car_seq: str):
    """상세 HTML에서 checkNo 추출 → M-park 성능점검 연결"""
    html = get_car_detail_html(car_seq)
    if not html:
        return None
    check_no = extract_mpark_checkno_from_html(html)
    if not check_no:
        print("M-park checkNo를 찾지 못했습니다.")
        return None
    print(f"M-park checkNo: {check_no}")
    return fetch_mpark_check_report(check_no)

# 테스트 실행 (첫 carSeq 기준)
if test_car_seqs:
    test_seq = test_car_seqs[0]
    print(f"\n=== M-park 성능점검기록부 테스트: carSeq {test_seq} ===")
    mpark = enrich_with_mpark_report(test_seq)
    if mpark:
        print("✅ M-park 성능점검 조회 성공")
        # 핵심 필드만 요약 출력
        keys = [
            'carName','oldCarNo','km','yymm','chkStrDay','chkEndDay','frameNo',
            'warrantyType','sagoGbn','enginChk','gearChk','remarks'
        ]
        for k in keys:
            print(f"{k}: {mpark.get(k)}")
    else:
        print("❌ 성능점검 정보를 가져오지 못했습니다.")


## 🧩 옵션 정보 추출 및 병합

상세페이지 HTML에서 옵션을 추출하고, 가능하면 옵션 전용 JSON 엔드포인트도 시도합니다.


In [ ]:
def try_fetch_options_api(car_seq):
    """옵션 전용 JSON 엔드포인트 시도 (있으면 사용)"""
    candidates = [
        "https://www.kbchachacha.com/public/car/detail/option/list.json",
        "https://www.kbchachacha.com/public/car/common/option/list.json",
        "https://www.kbchachacha.com/public/car/detail/option.v2.json",
    ]
    for url in candidates:
        try:
            r = session.post(url, data={"carSeq": str(car_seq)}, timeout=10)
            if r.status_code != 200:
                r = session.get(url, params={"carSeq": str(car_seq)}, timeout=10)
            if r.status_code == 200:
                data = r.json()
                if isinstance(data, dict) and (data.get("list") or data.get("options")):
                    return data.get("list") or data.get("options")
        except Exception:
            pass
    return None


def extract_options_from_html(detail_html):
    """상세 HTML에서 옵션 리스트 추출 (fallback)"""
    if not detail_html:
        return []
    soup = BeautifulSoup(detail_html, 'html.parser')
    options = []
    # 1) '옵션' 섹션 근처의 리스트 항목 추출
    # 사이트 구조에 따라 클래스/섹션명을 보강해야 함
    for section in soup.select('.option, .opt, .option-area, .option-list, .select-option'):
        for li in section.select('li'):
            txt = li.get_text(strip=True)
            if txt and len(txt) >= 2:
                options.append(txt)
    # 2) 텍스트 기반 백업: '옵션' 단어 주변 텍스트 토큰화
    if not options:
        text = soup.get_text("\n")
        if '옵션' in text:
            import re
            # 한국어/영문/숫자/괄호/+/,- 포함
            tokens = re.findall(r"[가-힣A-Za-z0-9+/()\-]{2,}", text)
            # 과도한 노이즈 제거 기본 필터링
            whitelist = ['썬루프','파노라마','통풍시트','열선시트','열선핸들','HUD','크루즈','스마트크루즈','차선이탈','후측방','어라운드뷰','하이패스','전자식파킹','오토홀드','전동트렁크','메모리시트','BOSE','JBL','B&O','네비','내비','LED','HID','DRL','뒷좌석열선','리어커튼','핸들열선','핸들통풍','통합주행모드','스마트키','원격시동','자동주차']
            for w in whitelist:
                if w in tokens:
                    options.append(w)
    # 중복 제거
    options = list(dict.fromkeys(options))
    return options


def enrich_with_options(car_seq):
    # 1) 옵션 API 먼저 시도
    data = try_fetch_options_api(car_seq)
    if data:
        # 다양한 구조 대응
        if isinstance(data, list):
            # [{name:..}, {optionName:..}] 등 케이스 처리
            names = []
            for x in data:
                if isinstance(x, dict):
                    names.append(x.get('name') or x.get('optionName') or x.get('title'))
                elif isinstance(x, str):
                    names.append(x)
            return [n for n in names if n]
    # 2) 실패 시 HTML 파싱
    html = get_car_detail_html(car_seq)
    return extract_options_from_html(html)

# 테스트: 옵션 추출
if test_car_seqs:
    test_seq = test_car_seqs[0]
    print(f"\n=== 옵션 추출 테스트: carSeq {test_seq} ===")
    opts = enrich_with_options(test_seq)
    print(f"옵션 수: {len(opts)}")
    print(opts[:20])
